# Streamlit code to bring together financial metrics

In [7]:
!pip install streamlit pandas # Install Streamlit and pandas
!pip install pyngrok # For ngrok tunnel, which is generally more reliable
!pip install plotly

In [8]:
!ngrok authtoken 2yxVXCrmtPGi226dvGWcPMHD8MI_73k2EmZ9zJ7io8xidgroS

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import os # Import os for path manipulation

# Hardcoded Regulatory/Industry Thresholds and Scoring Logic
# For simplicity, using a 5-point scale: 5 (Excellent) to 1 (Critical)
# Bands are exclusive on the upper bound: [lower, upper)

metric_info = {
    "Cash Ratio": {
        "description": "Indicates a bank's ability to cover its short-term liabilities with its most liquid assets. Often superseded by LCR for regulatory purposes. Values are typically ratios (e.g., 2.0x).",
        "type": "higher_better",
        "benchmark": 1.0, # Equivalent to LCR >= 100% or ratio >= 1.0
        "benchmark_display": "LCR >= 100% (or ratio >= 1.0)",
        "bands": { # Assuming actual values are ratios, e.g., 2.67 from your data
            (2.0, float('inf')): 5, # >2.0 (excellent liquidity)
            (1.5, 2.0): 4,          # 1.5-2.0 (good)
            (1.0, 1.5): 3,          # 1.0-1.5 (adequate, at/above benchmark)
            (0.7, 1.0): 2,          # 0.7-1.0 (concerning, below benchmark)
            (0, 0.7): 1             # <0.7 (critical)
        }
    },
    "Cost-to-Income Ratio": {
        "description": "Measures a bank's operational efficiency. Lower is generally better. Values are typically percentages (e.g., 55%).",
        "type": "lower_better",
        "benchmark": 60, # Industry benchmark (as a percentage)
        "benchmark_display": "< 60%",
        "bands": { # Assuming actual values are percentages, e.g., 56.48
            (0, 45): 5,
            (45, 55): 4,
            (55, 65): 3,
            (65, 75): 2,
            (75, float('inf')): 1
        }
    },
    "Current Ratio": {
        "description": "A general liquidity ratio, current assets to current liabilities. For banks, LCR/NSFR are more specific. Values are typically ratios.",
        "type": "higher_better",
        "benchmark": 1.0, # General benchmark for liquidity
        "benchmark_display": "> 1 (general corporate, for banks it's about robust liquidity management via LCR/NSFR)",
        "bands": { # Assuming actual values are ratios, e.g., 2.67
            (2.0, float('inf')): 5,
            (1.5, 2.0): 4,
            (1.0, 1.5): 3,
            (0.5, 1.0): 2,
            (0, 0.5): 1
        }
    },
    "Debt-to-Equity Ratio": {
        "description": "Measures a bank's leverage. Banks typically have higher ratios due to deposits. Regulators focus on capital ratios. Values are ratios.",
        "type": "lower_better",
        "benchmark": None, # No explicit regulatory max, but lower D/E implies less leverage.
        "benchmark_display": "No explicit regulatory max, but lower D/E implies less leverage (varies highly by bank type).",
        "bands": { # Banks can have high D/E. These bands are illustrative for ratios.
            (0, 15): 5, # Very strong capital base relative to debt (for banks, this is low leverage)
            (15, 30): 4,
            (30, 50): 3,
            (50, 70): 2,
            (70, float('inf')): 1
        }
    },
    "Equity-to-Assets Ratio": {
        "description": "Measures the proportion of assets financed by equity. Inverse of the leverage ratio (assets/equity). Values are typically decimals (e.g., 0.05 for 5%).",
        "type": "higher_better",
        "benchmark": 0.03, # Related to Leverage Ratio (Tier 1 Capital / Total Assets) >= 3%
        "benchmark_display": "Related to Leverage Ratio (Tier 1 Capital / Total Assets) >= 3% (Basel III)",
        "bands": {
            (0.05, float('inf')): 5, # > 5% Equity/Assets
            (0.04, 0.05): 4,         # 4-5% Equity/Assets
            (0.03, 0.04): 3,         # 3-4% Equity/Assets (at/above benchmark)
            (0.02, 0.03): 2,         # 2-3% Equity/Assets (below benchmark)
            (0, 0.02): 1             # < 2% Equity/Assets (critical)
        }
    },
    "Net Interest Margin (NIM)": {
        "description": "A key profitability measure, showing the difference between interest income and interest expense, relative to earning assets. Values are typically decimals (e.g., 0.02 for 2%).",
        "type": "higher_better",
        "benchmark": 0.03, # Generally 2-4% is considered healthy.
        "benchmark_display": "No regulatory min; generally 2-4% is considered healthy.",
        "bands": {
            (0.04, float('inf')): 5,
            (0.03, 0.04): 4,
            (0.02, 0.03): 3,
            (0.01, 0.02): 2,
            (float('-inf'), 0.01): 1
        }
    },
    "Non-Performing Loan (NPL) Ratio": {
        "description": "Measures asset quality and credit risk. Lower is better. Values are typically decimals (e.g., 0.03 for 3%).",
        "type": "lower_better",
        "benchmark": 0.05, # < 5% benchmark
        "benchmark_display": "< 5% (often aims for < 2-3% in healthy economies)",
        "bands": {
            (0, 0.01): 5,
            (0.01, 0.03): 4,
            (0.03, 0.05): 3,
            (0.05, 0.07): 2,
            (0.07, float('inf')): 1
        }
    },
    "Return on Assets (ROA)": {
        "description": "Measures how efficiently a bank uses its assets to generate profit. Values are typically decimals (e.g., 0.005 for 0.5%).",
        "type": "higher_better",
        "benchmark": 0.005, # Generally > 0.5%
        "benchmark_display": "No regulatory min; generally > 0.5% for healthy banks.",
        "bands": {
            (0.01, float('inf')): 5,
            (0.0075, 0.01): 4,
            (0.005, 0.0075): 3,
            (0, 0.005): 2,
            (float('-inf'), 0): 1
        }
    },
    "Return on Equity (ROE)": {
        "description": "Measures the return generated on shareholders' equity. High ROE can also signal high leverage. Values are typically decimals (e.g., 0.08 for 8%).",
        "type": "higher_better",
        "benchmark": 0.08, # Generally > 8-10%
        "benchmark_display": "No regulatory min; generally > 8-10% for healthy banks.",
        "bands": {
            (0.15, float('inf')): 5,
            (0.10, 0.15): 4,
            (0.08, 0.10): 3,
            (0, 0.08): 2,
            (float('-inf'), 0): 1
        }
    },
    "Tier 1 Capital Ratio": {
        "description": "A core measure of a bank's financial strength and ability to absorb losses. Values are typically decimals (e.g., 0.06 for 6%).",
        "type": "higher_better",
        "benchmark": 0.06, # >= 6% (Basel III minimum)
        "benchmark_display": ">= 6% (Basel III minimum for Tier 1 Capital)",
        "bands": {
            (0.08, float('inf')): 5, # > 8% (well-capitalized in many contexts)
            (0.07, 0.08): 4,
            (0.06, 0.07): 3,         # 6-7% (at/above regulatory minimum)
            (0.045, 0.06): 2,        # 4.5-6% (under-capitalized warning)
            (0, 0.045): 1            # < 4.5% (significantly under-capitalized)
        }
    }
}

def get_metric_score(metric_name, value):
    """Calculates a score (1-5) for a given metric value based on defined bands."""
    info = metric_info.get(metric_name)
    if not info or pd.isna(value):
        return None

    for (lower, upper), score in info["bands"].items():
        if info["type"] == "higher_better" and lower <= value < upper:
            return score
        elif info["type"] == "lower_better" and lower < value <= upper:
            return score
    return None

def calculate_quarterly_health(df_quarter):
    """Calculates an overall weighted health score for a given quarter."""
    total_weighted_score = 0
    total_weight = 0
    detailed_scores = {}

    metric_weights = {
        "Tier 1 Capital Ratio": 3,
        "Non-Performing Loan (NPL) Ratio": 2.5,
        "Equity-to-Assets Ratio": 2,
        "Cash Ratio": 1.5,
        "Net Interest Margin (NIM)": 1.5,
        "Return on Assets (ROA)": 1,
        "Return on Equity (ROE)": 1,
        "Cost-to-Income Ratio": 0.5,
        "Debt-to-Equity Ratio": 0.5,
        "Current Ratio": 0.5,
    }

    # Iterate over all defined metrics to ensure they always have an entry in detailed_scores
    for metric_name in metric_info.keys():
        row_data = df_quarter[df_quarter['Financial Metric Name'] == metric_name]

        # Initialize common details for the current metric
        current_metric_details = {
            "Actual Value Raw": float('nan'), # Default to NaN if not found in data
            "QoQ Change (%)": "N/A",
            "Base Score": "N/A",
            "Adjusted Score": "N/A",
            "Details": ""
        }

        if not row_data.empty: # If data for this metric exists in the current quarter's DataFrame
            actual_value_raw = row_data['Actual Value'].iloc[0]
            qoq_change = row_data['% Change (QoQ)'].iloc[0]

            current_metric_details["Actual Value Raw"] = actual_value_raw
            current_metric_details["QoQ Change (%)"] = f"{qoq_change:.2f}"

            # Only proceed to score if actual value is not NaN and metric has a weight
            if not pd.isna(actual_value_raw) and metric_weights.get(metric_name, 0) > 0:
                actual_value_for_scoring = actual_value_raw
                # Convert actual_value to match the scale of bands (e.g., 6.00% to 0.06 for ratios)
                if metric_name in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                                  "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)"]:
                    if actual_value_raw >= 1.0 and actual_value_raw < 100.0:
                         actual_value_for_scoring = actual_value_raw / 100.0

                current_metric_details["Actual Value Used"] = actual_value_for_scoring # Store scaled value for plotting

                metric_base_score = get_metric_score(metric_name, actual_value_for_scoring)

                if metric_base_score is not None:
                    change_adjustment = 0
                    if qoq_change > 10 and metric_info[metric_name]["type"] == "higher_better":
                        change_adjustment = 1
                    elif qoq_change < -10 and metric_info[metric_name]["type"] == "lower_better":
                        change_adjustment = 1
                    elif qoq_change < -10 and metric_info[metric_name]["type"] == "higher_better":
                        change_adjustment = -1
                    elif qoq_change > 10 and metric_info[metric_name]["type"] == "lower_better":
                        change_adjustment = -1

                    final_metric_score = max(1, min(5, metric_base_score + change_adjustment))
                    total_weighted_score += (final_metric_score * metric_weights.get(metric_name, 0))
                    total_weight += metric_weights.get(metric_name, 0)

                    current_metric_details.update({
                        "Base Score": metric_base_score,
                        "Adjusted Score": final_metric_score,
                        "Details": f"Weighted Score: {final_metric_score * metric_weights.get(metric_name, 0):.2f}"
                    })
                else:
                    current_metric_details["Details"] = f"Could not score value {actual_value_raw:.2f} within defined bands."
            else:
                current_metric_details["Details"] = "Missing actual value or zero weight assigned, skipping score calculation."
        else: # Metric not found in the quarter's data
            current_metric_details["Details"] = f"No data found for {metric_name} in the selected quarter."

        detailed_scores[metric_name] = current_metric_details

    overall_average_score = total_weighted_score / total_weight if total_weight > 0 else 0
    return overall_average_score, detailed_scores

def plot_qoq_chart(df_all_quarters, metric_name, metric_info_dict):
    """Plots the QoQ trend for a given metric across all available quarters."""
    df_metric = df_all_quarters[df_all_quarters['Financial Metric Name'] == metric_name].copy()
    if df_metric.empty:
        st.write(f"No data available for {metric_name} across quarters.")
        return

    df_metric['Year'] = df_metric['Year'].astype(int)
    df_metric['Quarter'] = df_metric['Quarter'].astype(int)
    df_metric = df_metric.sort_values(by=['Year', 'Quarter'])
    df_metric['Quarter_Label'] = df_metric['Year'].astype(str) + ' Q' + df_metric['Quarter'].astype(str)

    y_axis_title = "Value"
    # Logic to format value for plot based on metric type (percentage vs. absolute ratio)
    if metric_name in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                      "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)", "Cost-to-Income Ratio"]:
        df_metric['Value_for_Plot'] = df_metric['Actual Value'].apply(
            # Apply scaling only if original value is a small decimal and should be percentage
            lambda x: x * 100 if pd.notna(x) and x < 1.0 and metric_info_dict[metric_name]['benchmark'] is not None and metric_info_dict[metric_name]['benchmark'] < 1.0 else x
        )
        y_axis_title = "Value (%)"
    else: # For ratios like Cash Ratio, Current Ratio, Debt-to-Equity
        df_metric['Value_for_Plot'] = df_metric['Actual Value']
        y_axis_title = "Ratio" # More appropriate for these metrics

    fig = px.line(df_metric, x='Quarter_Label', y='Value_for_Plot',
                  title=f'{metric_name} Trend Over Quarters',
                  labels={'Value_for_Plot': y_axis_title, 'Quarter_Label': 'Quarter'},
                  markers=True)

    # Add benchmark line if available AND NOT Cash Ratio
    benchmark = metric_info_dict[metric_name].get("benchmark")
    if benchmark is not None and metric_name != "Cash Ratio": # <-- Modified: exclude Cash Ratio
        benchmark_for_plot = benchmark
        # Convert benchmark to plot scale if original benchmark is decimal and metric is percentage type
        if metric_name in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                          "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)"]:
            if benchmark < 1.0: # Only multiply by 100 if benchmark itself is a decimal (like 0.06)
                benchmark_for_plot = benchmark * 100

        fig.add_hline(y=benchmark_for_plot, line_dash="dash", line_color="grey",
                      annotation_text=f"Benchmark: {metric_info_dict[metric_name]['benchmark_display']}",
                      annotation_position="bottom right")

    fig.update_layout(hovermode="x unified")
    st.plotly_chart(fig, use_container_width=True)


# --- Streamlit UI ---
st.set_page_config(layout="wide", page_title="Bank Fiscal Health Dashboard")

st.title("Bank Fiscal Health Dashboard (QoQ Assessment)")
st.markdown("This dashboard provides a quantitative health measure for a bank based on key financial metrics and regulatory/industry benchmarks. Upload one or more files to get started.")

# File Uploader
uploaded_files = st.file_uploader("Upload your financial data file(s) (CSV or XLSX)", type=["csv", "xlsx"], accept_multiple_files=True)

processed_data = {} # Dictionary to store {tab_name: df}
tab_name_counts = {} # To handle duplicate tab names
all_bank_quarterly_scores = [] # Store scores for comparison tab

if uploaded_files:
    for uploaded_file in uploaded_files:
        filename = uploaded_file.name
        base_name = os.path.splitext(filename)[0]
        tab_base_name = base_name[:3].upper()

        current_counter = tab_name_counts.get(tab_base_name, 0)
        if current_counter > 0:
            current_tab_name = f"{tab_base_name}-{current_counter}"
        else:
            current_tab_name = tab_base_name
        tab_name_counts[tab_base_name] = current_counter + 1

        try:
            if filename.endswith('.csv'):
                df = pd.read_csv(uploaded_file)
            elif filename.endswith('.xlsx'):
                df = pd.read_excel(uploaded_file)
            else:
                st.warning(f"Skipping {filename}: Unsupported file type. Please upload CSV or XLSX.")
                continue

            required_columns = ['Year', 'Quarter', 'Financial Metric Name', 'Actual Value', '% Change (QoQ)']
            if not all(col in df.columns for col in required_columns):
                st.error(f"Error in {filename}: Missing required columns. Ensure it has: {', '.join(required_columns)}")
                continue

            df['YearQuarter'] = df['Year'].astype(str) + ' Q' + df['Quarter'].astype(str)
            processed_data[current_tab_name] = df

            # Calculate overall scores for ALL quarters for this bank for the comparison tab
            unique_quarters_in_file = df['YearQuarter'].unique()
            for quarter_label in unique_quarters_in_file:
                df_quarter_data = df[df['YearQuarter'] == quarter_label].copy()
                overall_score_q, _ = calculate_quarterly_health(df_quarter_data)
                year, quarter = int(quarter_label.split(' Q')[0]), int(quarter_label.split(' Q')[1])
                all_bank_quarterly_scores.append({
                    'bank_name': current_tab_name,
                    'year': year,
                    'quarter': quarter,
                    'overall_score': overall_score_q
                })

        except Exception as e:
            st.error(f"Error reading {filename}: {e}. Please ensure it's a valid CSV or XLSX file and matches the expected format.")

if processed_data:
    tab_names_sorted = sorted(processed_data.keys())
    tab_names_sorted.append("All Bank Comparison") # Add comparison tab name
    tabs = st.tabs(tab_names_sorted)

    for i, tab_name in enumerate(tab_names_sorted):
        with tabs[i]:
            if tab_name == "All Bank Comparison":
                st.header("Comparative Analysis Across All Banks")
                if not all_bank_quarterly_scores or len(processed_data) < 2:
                    st.info("Upload data for at least two banks to see a comparison.")
                else:
                    df_comparison = pd.DataFrame(all_bank_quarterly_scores)
                    df_comparison['Full_Quarter_Label'] = df_comparison['year'].astype(str) + ' Q' + df_comparison['quarter'].astype(str)
                    df_comparison = df_comparison.sort_values(by=['year', 'quarter'])

                    st.subheader("Overall Health Score Trend Across Banks")
                    fig_comp = px.line(df_comparison, x='Full_Quarter_Label', y='overall_score', color='bank_name',
                                       title="Overall Health Score Trend (1-5 Scale)",
                                       labels={'overall_score': 'Average Health Score (1-5)', 'Full_Quarter_Label': 'Quarter'},
                                       markers=True)
                    fig_comp.update_layout(hovermode="x unified")
                    st.plotly_chart(fig_comp, use_container_width=True)

                    st.subheader("Quarterly Health Score Summary Table")
                    # Ensure the pivot table includes all quarters present in the comparison data
                    pivot_df = df_comparison.pivot_table(index='Full_Quarter_Label', columns='bank_name', values='overall_score')
                    # Fill NaN (missing quarters for some banks) with 0 or a placeholder
                    pivot_df = pivot_df.fillna(0).sort_index(key=lambda x: [int(q.split(' Q')[0]) * 4 + int(q.split(' Q')[1]) for q in x])

                    st.dataframe(pivot_df.style.highlight_max(axis=1, color='lightgreen').highlight_min(axis=1, color='salmon'), use_container_width=True)

                    st.markdown("---")
                    st.subheader("Specific Metric Comparison for a Selected Quarter")

                    all_quarters_available = set()
                    for df_bank in processed_data.values():
                        all_quarters_available.update(df_bank['YearQuarter'].unique())
                    all_quarters_available_sorted = sorted(list(all_quarters_available), key=lambda x: (int(x.split(' Q')[0]), int(x.split(' Q')[1])))

                    if all_quarters_available_sorted:
                        selected_comp_quarter = st.selectbox("Select Quarter for Cross-Bank Metric Comparison:", all_quarters_available_sorted,
                                                             index=len(all_quarters_available_sorted)-1, key="comp_quarter_select")

                        metric_names_for_comp = list(metric_info.keys())
                        selected_comp_metric = st.selectbox("Select Metric for Comparison:", metric_names_for_comp, key="comp_metric_select")

                        comp_data_list = []
                        for bank_name, df_bank in processed_data.items():
                            df_quarter_data = df_bank[(df_bank['YearQuarter'] == selected_comp_quarter) & (df_bank['Financial Metric Name'] == selected_comp_metric)]
                            if not df_quarter_data.empty:
                                actual_value_raw = df_quarter_data['Actual Value'].iloc[0]
                                display_value = actual_value_raw
                                # Apply same scaling for display as in individual tabs for consistency
                                if selected_comp_metric in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                                                          "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)", "Cost-to-Income Ratio"]:
                                    if actual_value_raw < 1.0: # If it's a decimal like 0.06
                                        display_value = actual_value_raw * 100
                                elif selected_comp_metric in ["Cash Ratio", "Current Ratio", "Debt-to-Equity Ratio"]:
                                    display_value = actual_value_raw # Ratios are displayed as is

                                comp_data_list.append({
                                    'Bank': bank_name,
                                    'Metric Value': display_value,
                                    'QoQ Change (%)': df_quarter_data['% Change (QoQ)'].iloc[0]
                                })

                        if comp_data_list:
                            df_comp_metric = pd.DataFrame(comp_data_list)

                            # Determine y_axis_title for the comparison bar chart
                            y_axis_title_comp = "Value"
                            if selected_comp_metric in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                                                      "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)", "Cost-to-Income Ratio"]:
                                y_axis_title_comp = "Value (%)"
                            elif selected_comp_metric in ["Cash Ratio", "Current Ratio", "Debt-to-Equity Ratio"]:
                                y_axis_title_comp = "Ratio"

                            fig_metric_comp = px.bar(df_comp_metric, x='Bank', y='Metric Value',
                                                     title=f'{selected_comp_metric} Comparison for {selected_comp_quarter}',
                                                     labels={'Metric Value': y_axis_title_comp},
                                                     text='Metric Value')

                            # Add benchmark line if available and not cash ratio
                            benchmark = metric_info[selected_comp_metric].get("benchmark")
                            if benchmark is not None and selected_comp_metric != "Cash Ratio":
                                benchmark_for_plot = benchmark
                                if selected_comp_metric in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                                                          "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)"]:
                                    if benchmark < 1.0:
                                        benchmark_for_plot = benchmark * 100
                                # For Cost-to-Income Ratio, benchmark is already a percentage, no *100 needed
                                # For Cash/Current/Debt-to-Equity, benchmark is a ratio, no *100 needed

                                fig_metric_comp.add_hline(y=benchmark_for_plot, line_dash="dash", line_color="grey",
                                                          annotation_text=f"Benchmark: {metric_info[selected_comp_metric]['benchmark_display']}",
                                                          annotation_position="top right")

                            fig_metric_comp.update_traces(texttemplate='%{y:.2f}%' if y_axis_title_comp == "Value (%)" else '%{y:.2f}', textposition='outside')
                            fig_metric_comp.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
                            st.plotly_chart(fig_metric_comp, use_container_width=True)
                        else:
                            st.info(f"No data available for {selected_comp_metric} in {selected_comp_quarter} for comparison across banks.")
                    else:
                        st.info("No common quarters found for cross-bank metric comparison.")
            else: # Individual bank tab content
                df_current_bank = processed_data[tab_name]

                st.header(f"Analysis for {tab_name}")

                quarters = df_current_bank['YearQuarter'].unique()
                quarters_sorted = sorted(quarters, key=lambda x: (int(x.split(' Q')[0]), int(x.split(' Q')[1])))

                if not quarters_sorted:
                    st.warning("No quarterly data found in this file for analysis.")
                    continue

                selected_quarter = st.selectbox(f"Select Quarter for Analysis ({tab_name}):", quarters_sorted,
                                                index=len(quarters_sorted)-1, key=f"{tab_name}_quarter_select")

                df_selected_quarter = df_current_bank[df_current_bank['YearQuarter'] == selected_quarter].copy()

                overall_score, detailed_scores = calculate_quarterly_health(df_selected_quarter)

                fig = go.Figure(go.Indicator(
                    mode = "gauge+number+delta",
                    value = overall_score,
                    domain = {'x': [0, 1], 'y': [0, 1]},
                    title = {'text': f"Overall Health Score for {tab_name} ({selected_quarter})"},
                    delta = {'reference': 3, 'increasing': {'color': "green"}, 'decreasing': {'color': "red"}},
                    gauge = {
                        'axis': {'range': [1, 5], 'tickwidth': 1, 'tickcolor': "darkblue"},
                        'bar': {'color': "darkblue"},
                        'bgcolor': "white",
                        'borderwidth': 2,
                        'bordercolor': "gray",
                        'steps': [
                            {'range': [1, 2.5], 'color': "red"},
                            {'range': [2.5, 3.5], 'color': "gold"},
                            {'range': [3.5, 5], 'color': "green"}
                            ],
                        'threshold': {
                            'line': {'color': "black", 'width': 4},
                            'thickness': 0.75,
                            'value': overall_score}}))

                fig.update_layout(height=300, margin=dict(l=10, r=10, t=50, b=10))
                st.plotly_chart(fig, use_container_width=True)

                col_left, col_center, col_right = st.columns([1, 2, 1])
                with col_center:
                    st.metric(label=f"Overall Health Score (Avg. 1-5) for {tab_name}", value=f"{overall_score:.2f}")
                    if overall_score >= 4.5:
                        st.success("Overall Health: Excellent! (Strong performance across metrics)")
                    elif overall_score >= 3.5:
                        st.info("Overall Health: Good (Generally healthy, meeting most benchmarks)")
                    elif overall_score >= 2.5:
                        st.warning("Overall Health: Neutral/Watch (Some areas of concern, requires monitoring)")
                    else:
                        st.error("Overall Health: Concerning/Critical (Significant distress signals)")


                st.markdown("---")
                st.subheader("Detailed Metric Analysis for " + selected_quarter)

                metric_display_cols = st.columns(3)
                j = 0
                for metric_name in metric_info.keys():
                    with metric_display_cols[j % 3]:
                        with st.container(border=True):
                            st.markdown(f"**{metric_name}**")
                            st.markdown(f"<small>{metric_info[metric_name]['description']}</small>", unsafe_allow_html=True)
                            st.markdown(f"Benchmark: `{metric_info[metric_name]['benchmark_display']}`")

                            score_details = detailed_scores.get(metric_name, {})

                            if pd.isna(score_details.get("Actual Value Raw")):
                                st.metric(label="Actual Value", value="N/A", delta="N/A")
                                st.write(f"**Score:** N/A")
                                st.write(f"*{score_details.get('Details', 'Data not available.')}*")
                            else:
                                display_value_str = ""
                                actual_value_raw = score_details["Actual Value Raw"]

                                if metric_name in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                                                  "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)", "Cost-to-Income Ratio"]:
                                    if actual_value_raw < 1.0 and metric_info[metric_name]["benchmark"] is not None and metric_info[metric_name]["benchmark"] < 1.0:
                                        display_value_str = f"{actual_value_raw * 100:.2f}%"
                                    else:
                                        display_value_str = f"{actual_value_raw:.2f}%"
                                else:
                                    display_value_str = f"{actual_value_raw:.2f}"

                                delta_value = score_details.get("QoQ Change (%)")
                                delta_color = "normal"
                                if delta_value != "N/A":
                                    try:
                                        delta_num = float(delta_value.replace('%', ''))
                                        if metric_info[metric_name]["type"] == "higher_better":
                                            delta_color = "inverse" if delta_num < 0 else "normal"
                                        else:
                                            delta_color = "normal" if delta_num < 0 else "inverse"
                                    except ValueError:
                                        delta_color = "normal"

                                st.metric(label="Actual Value", value=display_value_str, delta=f"{delta_value}%", delta_color=delta_color)
                                st.write(f"**Derived Score (1-5):** {score_details.get('Adjusted Score')}")
                                if score_details.get("Details") != "" and "Weighted Score" not in score_details.get("Details", ""):
                                    st.markdown(f"<small>*{score_details.get('Details')}*</small>", unsafe_allow_html=True)
                    j += 1

                st.markdown("---")
                st.subheader(f"Quarter-over-Quarter Trends for {tab_name}")
                for metric_name in metric_info.keys():
                    plot_qoq_chart(df_current_bank, metric_name, metric_info)

else:
    st.info("Please upload one or more CSV or XLSX files to get started and see the analysis.")

Overwriting app.py


In [ ]:
from pyngrok import ngrok
import os

# Terminate any existing ngrok tunnels if you restart the cell
ngrok.kill()

# Use the authtoken from environment variable if set, or set it directly
# os.environ["NGROK_AUTH_TOKEN"] = "YOUR_NGROK_AUTH_TOKEN" # Uncomment and set if you didn't run Step 3

# Connect to ngrok (Streamlit typically runs on port 8501)
public_url = ngrok.connect(addr="8501")
print(f"Streamlit App URL: {public_url}")

# Run Streamlit in the background
!streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false &

Streamlit App URL: NgrokTunnel: "https://852d-34-21-15-80.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.21.15.80:8501

